In [ ]:
# Q: what is this for?
# google vision
!pip install --upgrade google-cloud-vision

In [ ]:
# Q: do we still need this if we import matplotlib already?
# matplotlib
!pip install matplotlib

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# Load customer and restaurant data from CSV files in Google Drive
# I just upload them to Google File withou creating any folder, so if you do so, these files should be read!s
customer_data = pd.read_csv("/content/drive/MyDrive/user_data.csv")
restaurant_data = pd.read_csv("/content/drive/MyDrive/restaruant-_data.csv")
drive.mount("/content/drive", force_remount=True)

Searching (Recognize Food in Image)

In [ ]:
# upload the credential file to use vision
from google.colab import files
uploaded = files.upload()

Saving insy660-18db02f3796a.json to insy660-18db02f3796a.json


In [ ]:
# set the Environment Variable
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'insy660-18db02f3796a.json'

In [ ]:
from google.colab import files
from google.cloud import vision_v1
from google.cloud.vision_v1 import types

def upload_image():
    # Prompt the user to upload an image file
    uploaded = files.upload()

    # Check if the user canceled the upload
    if not uploaded:
        return None

    # Get the binary data of the uploaded image
    image_data = next(iter(uploaded.values()))
    return image_data

def recognize_food():
    image_data = upload_image()

    # Check if the user canceled the upload
    if image_data is None:
        print("Upload was canceled.")
        return None

    client = vision_v1.ImageAnnotatorClient()
    image = types.Image(content=image_data)
    response = client.label_detection(image=image)
    result = []

    if response.label_annotations[0].description != 'Food':
        return "No food recognized", ()
    for label in response.label_annotations:
        result.append((label.description, label.score))
    return result

# Example usage
result = recognize_food()
if result:
    print(result)

Saving chili-cheese-nacho-final.jpg to chili-cheese-nacho-final.jpg
[('Food', 0.9831330180168152), ('Ingredient', 0.8922363519668579), ('Recipe', 0.8692439198493958), ('Fast food', 0.8643869757652283), ('Cuisine', 0.8108083605766296), ('Staple food', 0.7808915972709656), ('Romano cheese', 0.7805270552635193), ('Dish', 0.7721107602119446), ('Panucho', 0.7616492509841919), ('Produce', 0.7600836157798767)]


Recommendation (Meal Planning)

In [ ]:
# this function will be called within "meal_recommendation()" function while the customer wants recommendations for several meals.
def meal_planning(budget:int, occasion:str, notInclude:str, time:int, rating:float, num_meals:int):

  # Extract the upper bound from the 'Budget' column
  restaurant_data['UpperBound'] = restaurant_data['Budget Range'].str.extract(r'\$(\d+) - \$\d+', expand=False)

  # Convert the 'UpperBound' column to a numeric data type
  restaurant_data['UpperBound'] = restaurant_data['UpperBound'].astype(int)

  # filter restaurant based on the above user inputs
  qulifiedRestaurant = restaurant_data[(restaurant_data["UpperBound"] <= budget) & (restaurant_data["Suitable Occasions"].str.contains(occasion)) & (restaurant_data["Type of Cuisine"] != notInclude) & (restaurant_data["Average Waiting Time (min)"] <= time) & (restaurant_data["Rating"] >= rating)]
  print(qulifiedRestaurant)
  # make recommendations based on number of meals required
  plans = []
  for index, row in qulifiedRestaurant.iterrows():

    restaurant = row["Name"]
    recomendDish = random.choice(list(row["Budget Range Dishes"].keys()))
    price = row["Budget Range Dishes"][recomendDish]

    one_meal = [str(restaurant), str(recomendDish), "$", str(price)]
    plans.append(one_meal)

    # check if reaching the requested number of meals
    if len(plans) == num_meals:
      break

  return plans
meal_planning(30, "Dinner", "Korean", 30, 4.3, 1)



# this function is to recommend a meal when customer choose "I don’t know what I want to eat”.
def meal_recommendation():

  # greeting first
  customer_name = input("What's your name: ")
  print(f"Hello {customer_name}, welcome to your meal recommendation! Let's start with some questions.")

  # get user input for basic parameters
  budget = int(input("What's your maximum budget: "))
  occasion = input("What's your occasion (please enter one word only): ").capitalize()
  notInclude = input("Is there anything you know you do not want to eat today (please enter one type of cuisine): ").capitalize()
  waiting_time = int(input("What's your maximum expected time for waiting (please enter a number): "))
  rating = float(input("What's your minimum expected rating of the restaurant (out of 5): "))

  # ask if the customer wants a meal plan
  num_meals = int(input("How many meals you want for recommendations: "))

  # make recommendations from the filtered restaurants
  recomendations = meal_planning(budget, occasion, notInclude, waiting_time, rating, num_meals)

  for recomendation in recomendations:
    print(f"I would recommend {recomendation} for your {num_meals} days plan.")
    response = input("Are you happy with my recomendation (yes/no): ").lower()

    # ask for customers' satisfaction
    if response == "yes":
      print("Great, your meal is added to your cart!")
      break
    else:
      print("Alright! May I recommend again?")


Surprise food

In [ ]:
# Q: may need to edit your code based on reading file not from this dict
# dictionary containing named food dishes from different restaurants in Montreal
montreal_food_dishes = {
    "Schwartz's Deli": "Smoked Meat Sandwich",
    "St-Viateur Bagel": "Bagel with Cream Cheese",
    "Au Pied de Cochon": "Duck Confit",
    "Chez L'Epicier": "Lobster Roll",
    "La Banquise": "Poutine",
    "La Queue de Cheval": "Steak Frites",
    "Park Restaurant": "Sushi Platter",
    "Gnocchi": "Italian Gnocchi",
    "Le Club Chasse et Pêche": "Lamb Shank",
    "Le Bremner": "Oysters Rockefeller",
    "Pho Lien": "Pho",
    "L'Avenue": "Benedict Eggs",
    "L'Express": "Coq au Vin",
    "Café Cherrier": "Croque-Monsieur",
    "Le Majestique": "Lobster Bisque",
    "Le Mousso": "Tasting Menu",
    "Maison Publique": "Burger",
    "Patati Patata": "Poutine",
    "Restaurant Manitoba": "Bison Tartare",
    "Wilensky's": "Special Wilensky's Sandwich"
}



def get_user_input(prompt):
    while True:
        user_input = input(prompt).lower()
        if user_input == "yes" or user_input == "no":
            return user_input
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")

def select_random_dish():
    # Randomly select a restaurant and dish
    selected_restaurant = random.choice(list(montreal_food_dishes.keys()))
    selected_dish = montreal_food_dishes[selected_restaurant]
    return selected_dish, selected_restaurant

def surprise_dish():
    conversation_active = True  # Variable to track state of conversation
    while conversation_active:
        q1 = get_user_input("Are you interested in getting surprised today, yes/no? \n")
        if q1 == "yes":
            q2 = get_user_input("Great! I can surprise you with an UberEats’ surprise meal today. Is there anything you would like for me to take into consideration?\n")
            if q2 == "yes":
                selected_dish, selected_restaurant = select_random_dish()
                q3 = get_user_input(f"Great. I have selected {selected_dish} at {selected_restaurant} as a surprise for you! Are you happy with this selection?\n")
                if q3 == "yes":
                    print("Great. The bag has been added to your cart. Click on the checkout button to complete your purchase.")
                    conversation_active = False  # End conversation
                elif q3 == "no":
                    while True:
                        selected_dish, selected_restaurant = select_random_dish()
                        q4 = get_user_input(f"I have selected {selected_dish} at {selected_restaurant} as an alternative. Are you happy with this selection?\n")
                        if q4 == "yes":
                            print("Great. The bag has been added to your cart. Click on the checkout button to complete your purchase.")
                            conversation_active = False  # End conversation
                            break
                        elif q4 == "no":
                            continue
                        else:
                            print("Invalid input. Please enter 'yes' or 'no'.")
                else:
                    print("Invalid input. Please enter 'yes' or 'no'.")
            else:
                selected_dish, selected_restaurant = select_random_dish()
                q3 = get_user_input(f"Great. I have selected {selected_dish} at {selected_restaurant} as a surprise for you! Are you happy with this selection?\n")
                if q3 == "yes":
                    print("Great. The bag has been added to your cart. Click on the checkout button to complete your purchase.")
                    conversation_active = False  # End conversation
                elif q3 == "no":
                    while True:
                        selected_dish, selected_restaurant = select_random_dish()
                        q4 = get_user_input(f"I have selected {selected_dish} at {selected_restaurant} as an alternative. Are you happy with this selection?\n")
                        if q4 == "yes":
                            print("Great. The bag has been added to your cart. Click on the checkout button to complete your purchase.")
                            conversation_active = False  # End conversation
                            break
                        elif q4 == "no":
                            continue
                        else:
                            print("Invalid input. Please enter 'yes' or 'no'.")
                else:
                    print("Invalid input. Please enter 'yes' or 'no'.")
        elif q1 == "no":
            print("Thank you! Have a nice day.")
            conversation_active = False  # End conversation

Eats Wrapped

In [ ]:
# Chatbot responses
# "I would like to see my EatsWrapped."

intro_msg = "Let's check your EatsWrapped data! I'm here to show you your food delivery stats for this year. What would you like to know?"
options_msg = "You can ask me about the top cuisines, your order history, your most eaten dishes, your app usage time, or the number of orders you've placed."

def generate_visualization(data, x_label, y_label, title):
    plt.figure(figsize=(10, 6))
    plt.bar(data[x_label], data[y_label])
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def check_eats_wrapped(user_input):
    if "cuisine" in user_input:
        top_cuisines = customer_data["cuisine"].value_counts().head(10)
        generate_visualization(top_cuisines, "Cuisine", "Count", "Top 10 Cuisines of the Year")
        return "Here are the top 10 cuisines of the year!"
    elif "order history" in user_input:
        random_customer = random.choice(customer_data["customer_name"])
        random_restaurant = random.choice(restaurant_data["Name"])
        return f"You ordered {random.randint(1, 20)} times from {random_restaurant}."
    elif "eaten dishes" in user_input:
        top_dishes = customer_data["dish"].value_counts().head(3)
        generate_visualization(top_dishes, "Dish", "Count", "Top 3 Most Eaten Dishes")
        return "Here are the top 3 most eaten dishes!"
    elif "app usage time" in user_input:
        app_usage_time = random.randint(500, 1500)
        return f"You've spent approximately {app_usage_time} minutes browsing the app this year."
    elif "number of orders" in user_input:
        total_orders = customer_data["order_id"].nunique()
        return f"You've placed {total_orders} orders this year."
    else:
        return "I'm sorry, I didn't understand that. " + options_msg



*Chatbot* Interaction

In [ ]:
def main():
  # greating and let user choose from our four scenarios
  print("Chatbot: Hi, how can I help you today?")
  user_input = int(input("""Chatbot: Would you like to choose a service from below? (please enter a number)
                                    1. Searching for food
                                    2. I don't know what to eat today
                                    3. Surprise meal bag plz
                                    4. Check my EatsWrapped!"""))

  # call respective functions when users made choices or say bye if none is choosing
  if user_input == 1:
    # call searching (image recognizing) function
    pass
  elif user_input == 2:
    # call recommendation function
    meal_recommendation()
  elif user_input == 3:
    # call suprise meal function
    surprise_dish ()
  elif user_input == 4:
    # call EatsWrapped function
    print(intro_msg)
    print(options_msg)
    for _ in range(4):
        user_input1 = input("""Chatbot: What would you like to see? Please enter one of the following:
                                        Cuisine
                                        Order history
                                        Eaten dishes
                                        App usage time
                                        Number of orders""").lower()
        response = check_eats_wrapped(user_input1)
        print(response)
  else:
    print("Chatbot: Goodbye have a nice day!")

if __name__ == "__main__":
  main()
